In [ ]:
pip install requests

In [ ]:
pip install beautifulsoup4

In [ ]:
pip install lxml

In [ ]:
pip install selenium

In [ ]:
pip install openpyxl


# В данном проекте, мы напишем простенький парсер неденамического сайта, при этом мы создадим csv файл и заодно json файл для каждого продукта

In [54]:
import requests
import json
import csv
import random
from time import sleep 
from bs4 import BeautifulSoup

In [15]:
url = "https://health-diet.ru/table_calorie/?utm_source=leftMenu&utm_medium=table_calorie"

In [16]:
# Чуть-чуть шифруемся и делаем вид, что мы не бот).
headers = {
    "Accept": "**************************",
    "Accept-Encoding" : "**************************",
    "Accept-Language" : "**************************",
    "User-Agent" : "**************************"
}
# Можно еще добавить куки, но я в целях безопасности убрал их, чтобы залить этот проект на git


In [17]:
req = requests.get(url, headers = headers)

In [56]:
# Тут проверяем что ответ полученный от сервера 200, если так, то все супер, можно работать 
print(req)

<Response [200]>


In [70]:
src = req.text
# Копируем себе сайт, чтобы спокойно с ним работать
with open("index.html", "w", encoding = 'utf-8') as file: 
    file.write(src)

In [71]:
with open("index.html", encoding = 'utf-8') as file:
    scr = file.read()

soup = BeautifulSoup(src, 'lxml')
all_products_class = soup.find_all(class_ = 'mzr-tc-group-item-href')

all_categories = {}
for item in all_products_class:
    item_category = item.text
    item_link = "https://health-diet.ru" + item.get("href")

    all_categories[item_category] = item_link

In [72]:
for index, name in all_categories.items():
    print(f"{index} {name}")

In [ ]:
with open("all_categories.json", encoding = 'utf-8') as file:
    all_categories = json.load(file)

iteration_cnt = int(len(all_categories)) - 1
count = 0
print(f'Всего итераций {iteration_cnt}')
for category, category_link in all_categories.items():
    rep = [',', ' ', '-', "`", "'"]
    for item in rep:
        if item in category:
            category = category.replace(item, "_")

    req = requests.get(url = category_link, headers = headers)
    src = req.text

    with open(f"data/{count}_{category}.html", "w", encoding= "utf-8") as file:
        file.write(src)

    with open(f"data/{count}_{category}.html", encoding= "utf-8") as file:
        src = file.read()

    soup = BeautifulSoup(src, "lxml")

    alert_block = soup.find(class_ = "uk-alert-danger")
    if alert_block is not None:
        continue
    
    table_head = soup.find(class_='mzr-tc-group-table').find("tr").find_all("th")
    product = table_head[0].text
    calories = table_head[1].text
    proteins = table_head[2].text
    fats = table_head[3].text
    carbohydrates = table_head[4].text

    with open(f"data/{count}_{category}.csv", "w", encoding = "utf-8") as file:
        writer = csv.writer(file)
        writer.writerow([
            product,
            calories,
            proteins,
            fats,
            carbohydrates
        ])

    products_data = soup.find(class_ = "mzr-tc-group-table").find("tbody").find_all("tr")

    product_info = []
    for item in products_data:
        product_tds = item.find_all("td")

        title = product_tds[0].find("a").text
        calories = product_tds[1].text
        proteins = product_tds[2].text
        fats = product_tds[3].text
        carbohydrate = product_tds[4].text

        product_info.append(
            {
                "title" : title,
                "calories" : calories,
                "protein" : proteins,
                "fats" : fats,
                "carbohydrate" : carbohydrate
            }
        )
        
        with open(f"data/{count}_{category}.csv", "a", encoding = "utf-8") as file:
            writer = csv.writer(file)
            writer.writerow([
                title,
                calories,
                proteins,
                fats,
                carbohydrates
            ])

    with open(f'data/{count}_{category}.json', "a", encoding = "utf-8") as file:
        json.dump(product_info, file, indent = 4, ensure_ascii=False)
        
    count += 1
    print(f"Итерация {count}. {category} записан...")
    iteration_cnt -= 1

    if iteration_cnt == 0:
        print(f'Усе, работа off')
        break
        
    print(f'Осталось {iteration_cnt} итераций')
    sleep(random.randrange(2,4))

In [74]:
# Теперь у нас на компе создалась папка data/Название каждой категории и в 

True